# Benchmarking

In [ ]:
# pip install google-genai

In [1]:
from dotenv import load_dotenv
from tqdm import tqdm
import pandas as pd
import warnings
import logging
import os

warnings.filterwarnings("ignore")
logging.getLogger('transformers').setLevel(logging.ERROR)

load_dotenv()

True

In [2]:
Dataset = "Blended Skill Talk"             # Synthetic-PersonaChat, Blended Skill Talk, PEC, ConvAI2, FoCus, IT-ConvAI2

## Response Generation

In [3]:
# Loading the prompt
df = pd.read_csv(f'./Prompts/{Dataset}.csv')

df.head(10)

,personas,act_response,context
0,[User 1 persona]: ['i hate talking to people.'...,"I think it's because in my head, I think every...","User1: Wow, I am never shy. Do you have anxiet..."
1,[User 1 persona]: ['i have three daughters.' '...,What does your turtle eat? Is it hard to take...,User1: My turtle ran away from me today.\nUser...
2,[User 1 persona]: ['i hate the taste of fish.'...,"Yeah, kids grow up so quickly",User1: Our son in the Army is taking a leave t...
3,[User 1 persona]: ['my favorite movie is good ...,"Wow, you've done a marathon? I run a bit, but...","User1: that's awesome , i like running in the ..."
4,[User 1 persona]: ['my hair is black.' 'i like...,I would suggest a fitness place with a rock wa...,User1: Are there different skill levels? \nUse...
5,[User 1 persona]: ['my dad works at the mill a...,"I'm sure you'll do great. In second grade, tha...",User1: This is the first time I drop my kids o...
6,"[User 1 persona]: [""i'm going bald."" 'i ran in...",What do your kids look back at the pictures yo...,User1: I kept all of my kids first day pics ev...
7,[User 1 persona]: ['i like going to the moves....,"I get your feelings, but it's done and you mus...",User1: I fell bad about myself because I voted...
8,[User 1 persona]: ['i just started working as ...,"i've tall boots , that rock whether i like cou...",User1: My brother asked me if I was the one wh...
9,[User 1 persona]: ['i work in finance.' 'i hat...,"Thats cool, most shades of green are named aft...",User1: right on . i visited manhattan last sum...


In [4]:
# Function to generate a prompt
def create_prompt(personas, context, include_cot=True):

    prompt = (
        "I will provide you with a conversation context and the personas of the participants, that can be annotated with speaker information.\n"
        "As a participant in this conversation, your task is to generate a personalized response, considering the conversation context and personas.\n\n"
        "Participant Personas:\n"
        f"{personas}\n\n"
        "Conversation Context:\n"
        f"{context}\n\n"
        "Task Instruction:\n"
        "* Provide an unannotated response.\n"
        "* If only one persona is available, personalize the response accordingly.\n"
        "* If the conversation context is a single query, respond appropriately to the query.\n"
    )
    
    if include_cot:
        prompt += (
            "* Apply Chain of Thought reasoning to reflect on the alignment of your response with the personas.\n"
        )

    prompt += (
        "\nOutput Format: only give a JSON of the following format:\n"
        "{\n"
    )
    
    if include_cot:
        prompt += (
            '  "reasoning": "briefly describe your personalization process (in 110 words or less)."\n'
        )
        
    prompt += (
        '  "response": "provide the personalized natural language response here."\n'
        "}\n"
    )

    return prompt


first_row = df.iloc[204]
personas = first_row['personas']
context = first_row['context']

# Example usage
prompt = create_prompt(personas, context, include_cot=False)
print(prompt)


I will provide you with a conversation context and the personas of the participants, that can be annotated with speaker information.
As a participant in this conversation, your task is to generate a personalized response, considering the conversation context and personas.

Participant Personas:
[User 1 persona]: ["i'm a mother of 5." 'i like all types of music.']

Conversation Context:
User1: i do not find much time for it . i do not imagine you do either ?
User2: i actually like to craft when the kids are at school
User1: I think daycares are underrated. They let kids have so much expression and creativity and the employees should be paid more for that.

Task Instruction:
* Provide an unannotated response.
* If only one persona is available, personalize the response accordingly.
* If the conversation context is a single query, respond appropriately to the query.

Output Format: only give a JSON of the following format:
{
  "response": "provide the personalized natural language respons

### Gemini

In [6]:
g_api = os.getenv("GOOGLE_API_KEY")

In [7]:
COT_SETUP = False
LLM_name = "gemini-1.5-pro"  # gemini-1.5-pro

MAX_NEW_TOKEN = 220 if COT_SETUP else 110

In [8]:
from google import genai

client = genai.Client(api_key=g_api)

model = client.models


response = model.generate_content(
    model= LLM_name, 
    contents="You are a helpful response generator to output JSON.\n\n" + prompt,
    config={
        "temperature": 0,
        "max_output_tokens": MAX_NEW_TOKEN
    }
)


response_text = response.candidates[0].content.parts[0].text.strip().replace('```json', '').replace('```', '').strip()
response_text


'{\n  "response": "Between carpool, laundry, and meal prep, it feels like there\'s no time for anything!  I try to sneak in some music when I can, even if it\'s just in the car.  It\'s good to hear you get time to craft.  I used to love cross-stitch."\n}'

In [9]:
import json
response_data = json.loads(response_text)
response_data

{'response': "Between carpool, laundry, and meal prep, it feels like there's no time for anything!  I try to sneak in some music when I can, even if it's just in the car.  It's good to hear you get time to craft.  I used to love cross-stitch."}

In [10]:
import time
import json
import os
import pandas as pd
from tqdm import tqdm
from google import genai

client = genai.Client(api_key=g_api)
model = client.models

gen_responses = []
reasoning_responses = []
response_times = []

# CoT suffix
COT_ = "-COT" if COT_SETUP else ""

# Logging setup
log_dir = "./debugging logs"
os.makedirs(log_dir, exist_ok=True)
log_file_path = os.path.join(log_dir, f'{Dataset}_{LLM_name}_{COT_}.txt')

# Start logging and generation
with open(log_file_path, 'w') as log_file:
    for index, row in tqdm(df.iterrows(), total=len(df), desc="Generating responses"):
        personas = row['personas']
        context = row['context']

        prompt = create_prompt(personas, context, COT_SETUP)

        try:
            start_time = time.time()

            # Use system + user content format
            response = model.generate_content(
                model= LLM_name, 
                contents="You are a helpful response generator to output JSON.\n\n" + prompt,
                config={
                    "temperature": 0,
                    "max_output_tokens": MAX_NEW_TOKEN
                }
            )

            end_time = time.time()
            response_time = end_time - start_time
            
            response_text = response.candidates[0].content.parts[0].text.strip().replace('```json', '').replace('```', '').strip()

            # Remove markdown code blocks if present
    
            try:
                response_data = json.loads(response_text)
                gen_responses.append(response_data.get('response', ''))

                if COT_SETUP:
                    reasoning_responses.append(response_data.get('reasoning', ''))
                else:
                    reasoning_responses.append('')
            
            except json.JSONDecodeError as e:
                error_message = (f"[JSON ERROR] Index {index} — Failed to parse: {response_text}\n"
                                 f"Error: {str(e)}\n")
                print(f"[JSON ERROR] Index {index}")
                log_file.write(error_message + "\n")

                gen_responses.append(None)
                reasoning_responses.append(None)

            response_times.append(response_time)

        except Exception as e:
            error_message = f"[GENERATION ERROR] Index {index}: {str(e)}\n"
            print(f"[JSON ERROR] Index {index}")
            log_file.write(error_message + "\n")
            gen_responses.append(None)
            reasoning_responses.append(None)
            response_times.append(None)

# Construct and save final DataFrame
if COT_SETUP:
    response_df = pd.DataFrame({
        'gen_response': gen_responses,
        'reasoning': reasoning_responses,
        'response_time': response_times
    })
else:
    response_df = pd.DataFrame({
        'gen_response': gen_responses,
        'response_time': response_times
    })

# Save using consistent LLM_name-based filename
output_dir = f'./Responses/{Dataset}'
os.makedirs(output_dir, exist_ok=True)
response_df.to_csv(f'{output_dir}/{LLM_name}{COT_}.csv', index=False)

Generating responses:   5%|▍         | 47/980 [02:00<41:56,  2.70s/it]

[JSON ERROR] Index 46


Generating responses:   5%|▌         | 53/980 [02:55<1:26:34,  5.60s/it]

[JSON ERROR] Index 52


Generating responses:  13%|█▎        | 131/980 [06:14<34:21,  2.43s/it] 

...

Generating responses:  94%|█████████▍| 923/980 [42:02<03:09,  3.32s/it]

[JSON ERROR] Index 922


Generating responses: 100%|█████████▉| 979/980 [44:53<00:03,  3.12s/it]

[JSON ERROR] Index 978


Generating responses: 100%|██████████| 980/980 [44:57<00:00,  2.75s/it]


### Anthropic

In [ ]:
# pip install anthropic

In [7]:
COT_SETUP = False
LLM_name = "claude-sonnet-4-20250514"

MAX_NEW_TOKEN = 220 if COT_SETUP else 110

In [9]:
import anthropic
import os

client = anthropic.Anthropic(
    api_key= os.environ.get("ANTHROPIC_API_KEY"),
)
message = client.messages.create(
    model= LLM_name,
    max_tokens= MAX_NEW_TOKEN,
    system="You are a helpful response generator to output JSON.",
    
    messages=[
                {"role": "user", "content": prompt}
            ]
)
print(message.content)

[TextBlock(citations=None, text='{\n  "reasoning": "Based on the personas, I serve in the Army and find forts interesting, which makes me knowledgeable about military installations. I\'ve never been to NYC but wish to visit it. The user is asking about Fort Hamilton\'s current defensive status. As someone who serves in the Army and finds forts interesting, I would have knowledge about modern military installations and their purposes. I should provide informative details about Fort Hamilton\'s current role while expressing my personal interest in visiting it since it\'s in NYC, a', type='text')]


In [12]:
message

Message(id='msg_01SLpzvvnUd3khGirSdTW7PX', content=[TextBlock(citations=None, text='{\n  "reasoning": "Based on the personas, I serve in the Army and find forts interesting, which makes me knowledgeable about military installations. I\'ve never been to NYC but wish to visit it. The user is asking about Fort Hamilton\'s current defensive status. As someone who serves in the Army and finds forts interesting, I would have knowledge about modern military installations and their purposes. I should provide informative details about Fort Hamilton\'s current role while expressing my personal interest in visiting it since it\'s in NYC, a', type='text')], model='claude-sonnet-4-20250514', role='assistant', stop_reason='max_tokens', stop_sequence=None, type='message', usage=Usage(cache_creation=CacheCreation(ephemeral_1h_input_tokens=0, ephemeral_5m_input_tokens=0), cache_creation_input_tokens=0, cache_read_input_tokens=0, input_tokens=355, output_tokens=110, server_tool_use=None, service_tier='s

In [14]:
message.content[0].text

'{\n  "reasoning": "Based on the personas, I serve in the Army and find forts interesting, which makes me knowledgeable about military installations. I\'ve never been to NYC but wish to visit it. The user is asking about Fort Hamilton\'s current defensive status. As someone who serves in the Army and finds forts interesting, I would have knowledge about modern military installations and their purposes. I should provide informative details about Fort Hamilton\'s current role while expressing my personal interest in visiting it since it\'s in NYC, a'

In [ ]:
import time
import json
import os
import pandas as pd
from tqdm import tqdm
from google import genai

client = anthropic.Anthropic(
    api_key= os.environ.get("ANTHROPIC_API_KEY"),
)

gen_responses = []
reasoning_responses = []
response_times = []

# CoT suffix
COT_ = "-COT" if COT_SETUP else ""

# Logging setup
log_dir = "./debugging logs"
os.makedirs(log_dir, exist_ok=True)
log_file_path = os.path.join(log_dir, f'{Dataset}_{LLM_name}_{COT_}.txt')

# Start logging and generation
with open(log_file_path, 'w') as log_file:
    for index, row in tqdm(df.iterrows(), total=len(df), desc="Generating responses"):
        personas = row['personas']
        context = row['context']

        prompt = create_prompt(personas, context, COT_SETUP)

        try:
            start_time = time.time()

            response = client.messages.create(
                model= LLM_name,
                temperature=0,
                max_tokens= MAX_NEW_TOKEN,
                system="You are a helpful response generator to output JSON.",
                
                messages=[
                            {"role": "user", "content": prompt}
                        ]
            )

            end_time = time.time()
            response_time = end_time - start_time
            response_text = message.content[0].text

            try:
                response_data = json.loads(response_text)
                gen_responses.append(response_data.get('response', ''))

                if COT_SETUP:
                    reasoning_responses.append(response_data.get('reasoning', ''))
                else:
                    reasoning_responses.append('')
            
            except json.JSONDecodeError as e:
                error_message = (f"[JSON ERROR] Index {index} — Failed to parse: {response_text}\n"
                                 f"Error: {str(e)}\n")
                print(error_message)
                log_file.write(error_message + "\n")

                gen_responses.append(None)
                reasoning_responses.append(None)

            response_times.append(response_time)

        except Exception as e:
            error_message = f"[GENERATION ERROR] Index {index}: {str(e)}\n"
            print(error_message)
            log_file.write(error_message + "\n")
            gen_responses.append(None)
            reasoning_responses.append(None)
            response_times.append(None)

# Construct and save final DataFrame
if COT_SETUP:
    response_df = pd.DataFrame({
        'gen_response': gen_responses,
        'reasoning': reasoning_responses,
        'response_time': response_times
    })
else:
    response_df = pd.DataFrame({
        'gen_response': gen_responses,
        'response_time': response_times
    })

# Save using consistent LLM_name-based filename
output_dir = f'./Responses/{Dataset}'
os.makedirs(output_dir, exist_ok=True)
response_df.to_csv(f'{output_dir}/{LLM_name}{COT_}.csv', index=False)


Generating responses:   0%|          | 1/1000 [00:03<1:05:10,  3.91s/it]

[JSON ERROR] Index 0 — Failed to parse: {
  "reasoning": "Based on the personas, I serve in the Army and find forts interesting, which makes me knowledgeable about military installations. I've never been to NYC but wish to visit it. The user is asking about Fort Hamilton's current defensive status. As someone who serves in the Army and finds forts interesting, I would have knowledge about modern military installations and their purposes. I should provide informative details about Fort Hamilton's current role while expressing my personal interest in visiting it since it's in NYC, a
Error: Unterminated string starting at: line 2 column 16 (char 17)



Generating responses:   0%|          | 2/1000 [00:07<1:05:20,  3.93s/it]

[JSON ERROR] Index 1 — Failed to parse: {
  "reasoning": "Based on the personas, I serve in the Army and find forts interesting, which makes me knowledgeable about military installations. I've never been to NYC but wish to visit it. The user is asking about Fort Hamilton's current defensive status. As someone who serves in the Army and finds forts interesting, I would have knowledge about modern military installations and their purposes. I should provide informative details about Fort Hamilton's current role while expressing my personal interest in visiting it since it's in NYC, a
Error: Unterminated string starting at: line 2 column 16 (char 17)



Generating responses:   0%|          | 3/1000 [00:12<1:14:08,  4.46s/it]

...

[JSON ERROR] Index 11 — Failed to parse: {
  "reasoning": "Based on the personas, I serve in the Army and find forts interesting, which makes me knowledgeable about military installations. I've never been to NYC but wish to visit it. The user is asking about Fort Hamilton's current defensive status. As someone who serves in the Army and finds forts interesting, I would have knowledge about modern military installations and their purposes. I should provide informative details about Fort Hamilton's current role while expressing my personal interest in visiting it since it's in NYC, a
Error: Unterminated string starting at: line 2 column 16 (char 17)



Generating responses:   1%|▏         | 13/1000 [00:57<1:12:37,  4.41s/it]

[JSON ERROR] Index 12 — Failed to parse: {
  "reasoning": "Based on the personas, I serve in the Army and find forts interesting, which makes me knowledgeable about military installations. I've never been to NYC but wish to visit it. The user is asking about Fort Hamilton's current defensive status. As someone who serves in the Army and finds forts interesting, I would have knowledge about modern military installations and their purposes. I should provide informative details about Fort Hamilton's current role while expressing my personal interest in visiting it since it's in NYC, a
Error: Unterminated string starting at: line 2 column 16 (char 17)



Generating responses:   1%|▏         | 13/1000 [01:00<1:16:05,  4.63s/it]


KeyboardInterrupt: 

### OpenAI LLMs

In [ ]:
# pip install openai dotenv

In [ ]:
from dotenv import load_dotenv

load_dotenv()                   # Loading API key from .env file

In [ ]:
COT_SETUP = False
LLM_name = "gpt-4o-mini"  # gpt-3.5-turbo, gpt-4-turbo, gpt-4o-mini

MAX_NEW_TOKEN = 220 if COT_SETUP else 110

In [ ]:
import openai

client = openai.OpenAI()
        
# Generate a response from the model using the updated API
chat_completion = client.chat.completions.create(
    model= LLM_name, 
    messages=[
        # https://platform.openai.com/docs/guides/json-mode
        {"role": "system", "content": "You are a helpful response generator to output JSON."},
        {"role": "user", "content": prompt}
    ],
    temperature = 0,      # Based on FELM paper (Greedy Setup)
    max_tokens = MAX_NEW_TOKEN      # Based on max response length + reasoning
)

## Retrieve and return the response text
response_text = chat_completion.choices[0].message.content
response_text

'{\n  "reasoning": "I considered User 1\'s experience of loss and their participation in a book club, which suggests they might find solace in literature. I also acknowledged User 2\'s close relationship with their mom, which highlights the importance of family connections. My response aims to empathize with User 1\'s feelings while subtly suggesting that sharing experiences in a supportive environment, like a book club, could be beneficial during tough times.",\n  "response": "I can only imagine how tough it must be for you. It’s great that'

#### Actual Benchmarking

The error message you've provided indicates that the JSON string is malformed due to a missing comma (,) between the "reasoning" and "response" keys.

In [ ]:
import openai
import time
import re
import json
import os
from tqdm import tqdm
import pandas as pd

# Initialize OpenAI client once
client = openai.OpenAI()

gen_responses = []
reasoning_responses = []
response_times = []

# Determine COT_ value based on COT_SETUP
COT_ = "-COT" if COT_SETUP else ""

# Ensure the debugging logs directory exists
log_dir = "./debugging logs"
os.makedirs(log_dir, exist_ok=True)

# Prepare the log file path
log_file_path = os.path.join(log_dir, f'{Dataset}_{LLM_name}_{COT_}.txt')

# Open the log file for writing
with open(log_file_path, 'w') as log_file:

    for index, row in tqdm(df.iterrows(), total=len(df), desc="Generating responses"):
        personas = row['personas']
        context = row['context']

        prompt = create_prompt(personas, context, COT_SETUP)

        # Measure the start time
        start_time = time.time()

        # Generate a response from the model using the updated API
        chat_completion = client.chat.completions.create(
            model=LLM_name,
            messages=[
                {"role": "system", "content": "You are a helpful response generator to output JSON."},
                {"role": "user", "content": prompt}
            ],
            temperature=0,  # Based on FELM paper (Greedy Setup)
            max_tokens= MAX_NEW_TOKEN  # Based on max response length + reasoning
        )

        # Measure the end time and calculate the duration
        end_time = time.time()
        response_time = end_time - start_time

        # Retrieve and return the response text
        response_text = chat_completion.choices[0].message.content

        # # Clean the response to ensure proper JSON parsing
        # cleaned_response = response_text.replace('\n', ' ').replace('\r', '').strip().strip('"')

        # # # Attempt to automatically fix missing commas in JSON
        # # cleaned_response = re.sub(r'\"\s*\"', '", "', cleaned_response)

        # # # Check if the response ends with '}' and attempt to fix if not
        # # if not cleaned_response.endswith('}'):
        # #     cleaned_response += '}'

        try:
            response_data = json.loads(response_text)
            gen_responses.append(response_data.get('response', ''))

            if COT_SETUP:
                reasoning_responses.append(response_data.get('reasoning', ''))
            else:
                reasoning_responses.append('')  # Fill with empty string if COT_SETUP is False
        
        except json.JSONDecodeError as e:
            error_message = (f"Failed to parse JSON for response from index {index}: {response_text}\n"
                             f"Error: {str(e)}\n")
            print(error_message)
            log_file.write(error_message + "\n")


            # gen_responses.append('Error parsing response')
            # reasoning_responses.append('Error parsing response')

            gen_responses.append(None)
            reasoning_responses.append(None)


        # Store the response time
        response_times.append(response_time)

# Create the response DataFrame based on the COT_SETUP value
if COT_SETUP:
    response_df = pd.DataFrame({
        'gen_response': gen_responses,
        'reasoning': reasoning_responses,
        'response_time': response_times
    })
else:
    response_df = pd.DataFrame({
        'gen_response': gen_responses,
        'response_time': response_times
    })
 
# Save the response DataFrame to a CSV file
response_df.to_csv(f'./Responses/{Dataset}/{LLM_name}{COT_}.csv', index=False)


Generating responses:   1%|          | 12/1000 [00:15<23:31,  1.43s/it]

Failed to parse JSON for response from index 11: {
  "response": "Tolay Lake is a fascinating place with a rich history and diverse ecosystem. It's not just a beautiful spot in the Sonoma Mountains, but it also serves as a vital habitat for various species. The lake is home to several breeding pairs of golden eagles, which are truly majestic birds. Additionally, you can find other unique species like the California red-legged frog and the Western pond turtle. The area is also significant for its archaeological value, reflecting the history of Native American settlements. If you're interested in owning a park, Tol
Error: Unterminated string starting at: line 2 column 15 (char 16)



Generating responses:   1%|▏         | 14/1000 [00:18<22:52,  1.39s/it]

Failed to parse JSON for response from index 13: {
  "response": "Absolutely! The Inca road system, known as the Qhapaq Ñan, was not only vital for military movements but also for trade and communication across the vast empire. It connected various regions, allowing for the efficient transport of goods and troops. The roads were built with impressive engineering techniques, including stone-paved paths and suspension bridges. Additionally, the Incas established way stations along the routes, which provided rest and supplies for travelers. It's fascinating how this infrastructure supported both the economic and military strength of the In
Error: Unterminated string starting at: line 2 column 15 (char 16)



Generating responses:   2%|▏         | 18/1000 [00:23<21:41,  1.33s/it]

...

Generating responses:  92%|█████████▏| 916/1000 [21:59<01:58,  1.41s/it]

Failed to parse JSON for response from index 915: {
  "response": "Another interesting fact about Van Cortlandt Park is that it has a rich history beyond the Wappinger people. The park was established in 1888 and is one of the largest parks in New York City. It also features the Van Cortlandt House Museum, which is the oldest house in the Bronx, dating back to the 18th century. Additionally, the park is home to a variety of wildlife, making it a great spot for birdwatching, so you might just spot that woodpecker you're
Error: Unterminated string starting at: line 2 column 15 (char 16)



Generating responses:  98%|█████████▊| 975/1000 [23:16<00:32,  1.30s/it]

Failed to parse JSON for response from index 974: {
  "response": "In addition to the blue whale skeleton, the Beaty Biodiversity Museum has an impressive fossil collection that includes various prehistoric animal fossils. You can find fossils of ancient marine reptiles, like ichthyosaurs and plesiosaurs, which are fascinating to see. There are also dinosaur fossils, including some from the Cretaceous period, which showcase the diversity of life that once roamed the Earth. The museum often features rotating exhibits, so you might even catch a glimpse of some rare finds or special displays related to prehistoric life during
Error: Unterminated string starting at: line 2 column 15 (char 16)



Generating responses: 100%|██████████| 1000/1000 [23:51<00:00,  1.43s/it]


In [9]:
response_df.head(20)

,gen_response,response_time
0,Nazareth House has a rich history that reflect...,1.281389
1,The Truman Galusha House served as a gathering...,1.123755
2,The Marion Palace Theatre is a stunning exampl...,1.342017
3,Technische Universität Darmstadt is highly reg...,1.298877
4,"In addition to St Botolph's Church, you might ...",1.497569
5,Oakland University offers a wide range of unde...,1.484201
6,The Westin St. Francis is owned by Marriott In...,0.943593
7,In addition to the Chaos Crags and Crags Lake ...,1.652008
8,"Before its establishment, New Brunswick Theolo...",0.940908
9,The National War Memorial was officially opene...,1.339482
